In [3]:
import random
from collections import defaultdict, Counter
import numpy as np
import polars as pl
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
print(f'numpy version: {np.__version__}')
print(f'polars version: {pl.__version__}')

numpy version: 2.2.2
polars version: 1.24.0


In [4]:
SEED = 1 # random seed to ensure reproducibility
DATA_LOCATION = 'Mimic3_Data'

In [5]:
df_chartevents = pl.read_parquet(f'{DATA_LOCATION}/chartevents_trust.parquet')
print(df_chartevents.shape)
#print(df_chartevents.head(5))

(2852933, 3)


In [6]:
df_noteevents = pl.read_parquet(f'{DATA_LOCATION}/noteevents_trust.parquet')
print(df_noteevents.shape)

(2082294, 6)


In [7]:
chartevents_features = {}
grouped = df_chartevents.group_by('HADM_ID')
for hadm_id, rows in grouped:
    feats = {}
    for row in rows.iter_rows():
        label = row[1].lower()
        row_value = row[2]
        if row_value is None:
            val = 'none'
        else:
            val = row_value.lower()
        
        if 'reason for restraint' in label:       
            
            if (val == 'not applicable') or (val == 'none'):
                val = 'none'
            elif ('threat' in val) or ('acute risk of' in val):
                val = 'threat of harm'
            elif ('confusion' in val) or ('delirium' in val) or (val == 'impaired judgment') or (val == 'sundowning'):
                val = 'confusion/delirium'
            elif ('occurence' in val) or (val == 'severe physical agitation') or (val == 'violent/self des'):
                val = 'prescence of violence'
            elif (val == 'ext/txinterfere') or (val == 'protection of lines and tubes') or (val == 'treatment interference'):
                val = 'treatment interference'
            elif 'risk for fall' in val:
                val = 'risk for falls'
            else:
                val = val
            
            feats[('reason for restraint', val)] = 1

        elif 'restraint location' in label:
                
            if val == 'none':
                val = 'none'
            elif '4 point rest' in val:
                val = '4 point restraint'
            else:
                val = 'some restraint'
            
            feats[('restraint location', val)] = 1

        elif 'restraint device' in label:
                
            if 'sitter' in val:
                val = 'sitter'
            elif 'limb' in val:
                val = 'limb'
            else:
                val = val
            
            feats[('restraint device', val)] = 1
            
        elif 'bath' in label:
            if 'part' in label:
                val = 'partial'
            elif 'self' in val:
                val = 'self'
            elif 'refused' in val:
                val = 'refused'
            elif 'shave' in val:
                val = 'shave'
            elif 'hair' in val:
                val = 'hair'
            elif 'none' in val:
                val = 'none'
            else:
                val = 'done'
            
            feats[('bath', val)] = 1

        elif label in ['behavior', 'behavioral state']:
            #feats[('behavior', val)] = 1
            pass
            
        elif label.startswith('pain level'):
            feats[('pain level', val)] = 1
            
        elif label.startswith('pain management'):
            #feats[('pain management', val)] = 1
            pass
        elif label.startswith('pain type'):
            #feats[('pain type', val)] = 1
            pass
        elif label.startswith('pain cause'):
            #feats[('pain cause', val)] = 1
            pass
        elif label.startswith('pain location'):
            #feats[('pain location', val)] = 1
            pass
            
        elif label.startswith('education topic'):
            feats[('education topic', val)] = 1

        elif label.startswith('safety measures'):
            feats[('safety measures', val)] = 1
            
        elif label.startswith('side rails'):
            feats[('side rails', val)] = 1
    
        elif label.startswith('status and comfort'):
            feats[('status and comfort', val)] = 1

        elif 'informed' in label:
            feats[('informed', val)] = 1        
        else:

            if type(row_value) == type(''):
                # extract phrase
                featname = (label, row_value.lower())
                value = 1.0
                feats[featname] = value
            elif row_value is None:
                featname = (label,'none')
                value = 1.0
                feats[featname] = value
            else: 
                featname = (label,)
                value = value
                feats[featname] = value
                pass
    
    chartevents_features[hadm_id] = feats

all_hadm_ids = set(df_chartevents['HADM_ID'].unique())
mistrust_ids = set([])
count = 0
for hadm_id, rows in df_noteevents.group_by('HADM_ID'):
    # This is customizable to various note-based definitions of what to look for
    mistrust = False
    for row in rows.iter_rows():
        text = row[2].lower()
        if 'noncompliance' in text:
            mistrust = True
    # add the ID
    if mistrust:
        if isinstance(hadm_id, tuple):
            if len(hadm_id) > 1:
                raise ValueError('hadm_id is weird tuple', hadm_id)
            hadm_id = hadm_id[0]
        if hadm_id is not None:
            mistrust_ids.add(int(hadm_id))

# binary labels
trust_labels_noncompliance = {hadm_id:'trust' for hadm_id in all_hadm_ids}
for hadm_id in mistrust_ids:
    trust_labels_noncompliance[hadm_id] = 'mistrust'
    
print('patients labeled as trustful:', len([y for y in trust_labels_noncompliance.values() if y=='trust']))
print('patients labeled as mistrustful:', len([y for y in trust_labels_noncompliance.values() if y=='mistrust']))

vect = DictVectorizer()
vect.fit(chartevents_features.values())
print('num_features:', len(vect.get_feature_names_out()))

trust_Y_vect = {'mistrust': 1, 'trust': 0}
print(trust_Y_vect)

patients labeled as trustful: 53915
patients labeled as mistrustful: 611
num_features: 620
{'mistrust': 1, 'trust': 0}


In [8]:
autopsy_consent = []
autopsy_decline = []

grouped = df_noteevents.group_by('HADM_ID')
for hadm_id, rows in grouped:
    if isinstance(hadm_id, tuple):
        if len(hadm_id) > 1:
            raise ValueError('hadm_id is weird tuple', hadm_id)
        hadm_id = hadm_id[0]
    if hadm_id is not None:
        mistrust_ids.add(int(hadm_id))
    consented = False
    declined = False
    for row in rows.iter_rows():
        text = row[2].lower()
        for line in text.lower().split('\n'):
            if 'autopsy' in line:
                if 'decline' in line:
                    declined = True
                if 'not consent' in line:
                    declined = True
                if 'refuse' in line:
                    declined = True
                if 'denied' in line:
                    declined = True
                if 'consent' in line:
                    consented = True
                if 'agree' in line:
                    consented = True
                if 'request' in line:
                    consented = True
                
    # probably some "declined donation but consented to autopsy" or something confusing. just ignore hard cases
    if consented and declined:
        continue

    if consented:
        autopsy_consent.append(hadm_id)
    if declined:
        autopsy_decline.append(hadm_id)
        
# binary labels
trust_labels_autopsy = {}
for hadm_id in autopsy_consent:
    trust_labels_autopsy[int(hadm_id)] = 'mistrust'
for hadm_id in autopsy_decline:
    trust_labels_autopsy[int(hadm_id)] = 'trust'

print('patients labeled as trustful:', len([y for y in trust_labels_autopsy.values() if y=='trust']))
print('patients labeled as mistrustful:', len([y for y in trust_labels_autopsy.values() if y=='mistrust']))

patients labeled as trustful: 739
patients labeled as mistrustful: 270


In [9]:
chartevents_features = {k[0]: v for k, v in chartevents_features.items()}

In [10]:
def data_split(ids, ratio=0.7, seed=SEED):
    random.seed(seed)
    random.shuffle(ids)
    train = ids[:int(len(ids)*ratio)]
    test  = ids[int(len(ids)*ratio):]
    return train, test

def compute_stats_binary(task, pred, probas, ref, labels_map):
    """
    Core function for calculating binary classification metrics.
    
    Parameters:
        task (str): Task description/identifier
        pred (np.ndarray): Predicted labels (0 or 1)
        probas (np.ndarray): Model confidence scores for positive class
        ref (np.ndarray): Ground truth labels
        labels_map (dict): Mapping of label indices to class names
        
    Returns:
        dict: Dictionary containing accuracy, precision, recall, f1, 
              auc, sensitivity, and specificity
    """
    # Validate predictions match probability thresholds
    assert np.array_equal(pred, (probas > 0).astype(int)), "Predictions mismatch with probability thresholds"
    
    # Calculate confusion matrix using sklearn
    conf = confusion_matrix(ref, pred, labels=[0, 1])
    tn, fp, fn, tp = conf.ravel()
    
    # Compute metrics using sklearn functions
    precision = precision_score(ref, pred, zero_division=0)
    recall = recall_score(ref, pred, zero_division=0)
    f1 = f1_score(ref, pred, zero_division=0)
    accuracy = accuracy_score(ref, pred)
    specificity = tn / (tn + fp + 1e-9)
    
    # Calculate AUC if applicable
    auc = roc_auc_score(ref, probas) if len(np.unique(ref)) == 2 else None
    
    # Print formatted results
    print(f"\nTask: {task}")
    print(f"Confusion Matrix:\n{conf}")
    print(f"Specificity:\t{specificity:.3f}")
    print(f"Sensitivity:\t{recall:.3f}")  # Sensitivity = Recall
    if auc is not None:
        print(f"AUC:\t\t{auc:.3f}")
    print(f"Accuracy:\t{accuracy:.3f}")
    print(f"Precision:\t{precision:.3f}")
    print(f"F1 Score:\t{f1:.3f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'sensitivity': recall,
        'specificity': specificity
    }

def classification_results(svm, labels_map, X, Y, task):
    """
    Generates classification results from an SVM model.
    
    Parameters:
        svm: Trained SVM classifier
        labels_map (dict): Label index to name mapping
        X (np.ndarray): Feature matrix
        Y (np.ndarray): True labels
        task (str): Task description/identifier
        
    Returns:
        dict: Classification metrics
    """
    decision_scores = svm.decision_function(X)
    
    # Handle binary classification formatting
    if len(labels_map) == 2:
        probas = np.column_stack([-decision_scores, decision_scores])
    else:
        probas = decision_scores
        
    pred = probas.argmax(axis=1)
    return compute_stats_binary(task, pred, probas[:, 1], Y, labels_map)

# display informative features

def classification_analyze(task: str, vect, clf, labels_map: dict, num_feats=10):
    """
    Print the most positive and negative features for a binary linear classifier.

    Parameters:
        task (str): Name or description of the classification task.
        vect: Fitted vectorizer with a `vocabulary_` attribute.
        clf: Trained linear classifier (must have `coef_` attribute).
        labels_map (dict): Mapping from label names to integer indices.
        num_feats (int): Number of top features to display for each class.
    """
    # Map feature indices back to feature names
    ind2feat = {i: str(f) for f, i in vect.vocabulary_.items()}

    # Get sorted label names by their index
    sorted_labels = [label for label, i in sorted(labels_map.items(), key=lambda t: t[1])]
    if len(sorted_labels) != 2:
        raise ValueError("This function is for binary classification only.")

    coef = clf.coef_[0]
    sorted_indices = np.argsort(coef)

    # Most negative features (class 0)
    neg_indices = sorted_indices[:num_feats]
    # Most positive features (class 1)
    pos_indices = sorted_indices[-num_feats:][::-1]

    print(f"\nTop {num_feats} features for class '{sorted_labels[1]}' (positive):")
    for idx in pos_indices:
        val = coef[idx]
        word = ind2feat.get(idx, f"<UNK-{idx}>")
        if val > 1e-4:
            print(f"  {word:<25}: {val:7.4f}")

    print(f"\nTop {num_feats} features for class '{sorted_labels[0]}' (negative):")
    for idx in neg_indices:
        val = coef[idx]
        word = ind2feat.get(idx, f"<UNK-{idx}>")
        if val < -1e-4:
            print(f"  {word:<25}: {val:7.4f}")

    print("\n")

In [11]:
# CLASSIFIER: noncompliance

noncompliance_cohort = list(set(trust_labels_noncompliance.keys()) & all_hadm_ids)
print('patients:', len(noncompliance_cohort))

# train/test split
noncompliance_train_ids, noncompliance_test_ids = data_split(noncompliance_cohort)

# select pre-computed features
noncompliance_train_features = [chartevents_features[hadm_id] for hadm_id in noncompliance_train_ids]
noncompliance_test_features  = [chartevents_features[hadm_id] for hadm_id in noncompliance_test_ids ]

# vectorize features
noncompliance_train_X = vect.transform(noncompliance_train_features)
noncompliance_test_X  = vect.transform(noncompliance_test_features)

# select labels
noncompliance_train_Y = [trust_Y_vect[trust_labels_noncompliance[hadm_id]] for hadm_id in noncompliance_train_ids]
noncompliance_test_Y = [trust_Y_vect[trust_labels_noncompliance[hadm_id]] for hadm_id in noncompliance_test_ids]

# fit model
noncompliance_svm = LogisticRegression(C=0.1, penalty='l2', tol=0.01, random_state=SEED)
noncompliance_svm.fit(noncompliance_train_X, noncompliance_train_Y)
print(noncompliance_svm)

# evaluate model
classification_results(noncompliance_svm, trust_Y_vect, noncompliance_train_X, noncompliance_train_Y, 'train: noncompliance')
classification_results(noncompliance_svm, trust_Y_vect,  noncompliance_test_X,  noncompliance_test_Y, 'test: noncompliance')

# most informative features
classification_analyze('noncompliance', vect, noncompliance_svm, trust_Y_vect, num_feats=15)


patients: 54510
LogisticRegression(C=0.1, random_state=1, tol=0.01)

Task: train: noncompliance
Confusion Matrix:
[[37738     0]
 [  419     0]]
Specificity:	1.000
Sensitivity:	0.000
AUC:		0.554
Accuracy:	0.989
Precision:	0.000
F1 Score:	0.000

Task: test: noncompliance
Confusion Matrix:
[[16177     0]
 [  176     0]]
Specificity:	1.000
Sensitivity:	0.000
AUC:		0.508
Accuracy:	0.989
Precision:	0.000
F1 Score:	0.000

Top 15 features for class 'mistrust' (positive):
  ('support systems', 'parents'):  0.0169
  ('reason for restraint', 'confusion/delirium'):  0.0122
  ('non-violent restraints', 'off'):  0.0119
  ('social work consult', '1'):  0.0106
  ('judgement', 'impaired'):  0.0104
  ('education topic', 'diabetic'):  0.0099
  ('safety measures', '1:1 time with patient'):  0.0097
  ('side rails', 'all rails up - specialty beds only'):  0.0092
  ('restraint device', '4 side rails'):  0.0082
  ('restraints evaluated_v2', 'continued'):  0.0080
  ('reason for restraint', 'risk for falls'): 

In [12]:
# CLASSIFIER: autopsy

autopsy_cohort = list(set(trust_labels_autopsy.keys()) & set(df_chartevents['HADM_ID'].unique()))
print('patients:', len(autopsy_cohort))

# train/test split
autopsy_train_ids, autopsy_test_ids = data_split(autopsy_cohort)

# select pre-computed features
autopsy_train_features = [chartevents_features[hadm_id] for hadm_id in autopsy_train_ids]
autopsy_test_features  = [chartevents_features[hadm_id] for hadm_id in autopsy_test_ids ]

# vectorize features
autopsy_train_X = vect.transform(autopsy_train_features)
autopsy_test_X  = vect.transform(autopsy_test_features)

# select labels
autopsy_train_Y = [trust_Y_vect[trust_labels_autopsy[hadm_id]] for hadm_id in autopsy_train_ids]
autopsy_test_Y  = [trust_Y_vect[trust_labels_autopsy[hadm_id]] for hadm_id in autopsy_test_ids ]

# fit model
autopsy_svm = LogisticRegression(C=0.1, penalty='l2', tol=0.01, random_state=SEED)
autopsy_svm.fit(autopsy_train_X, autopsy_train_Y)
print(autopsy_svm)

# evaluate model
classification_results(autopsy_svm, trust_Y_vect, autopsy_train_X, autopsy_train_Y, 'train: autopsy')
classification_results(autopsy_svm, trust_Y_vect,  autopsy_test_X,  autopsy_test_Y, 'test:  autopsy')

# most informative features
classification_analyze('trust', vect, autopsy_svm, trust_Y_vect, num_feats=15)

patients: 997
LogisticRegression(C=0.1, random_state=1, tol=0.01)

Task: train: autopsy
Confusion Matrix:
[[498  12]
 [138  49]]
Specificity:	0.976
Sensitivity:	0.262
AUC:		0.850
Accuracy:	0.785
Precision:	0.803
F1 Score:	0.395

Task: test:  autopsy
Confusion Matrix:
[[204  17]
 [ 73   6]]
Specificity:	0.923
Sensitivity:	0.076
AUC:		0.550
Accuracy:	0.700
Precision:	0.261
F1 Score:	0.118

Top 15 features for class 'mistrust' (positive):
  ('pain level', 'unable to score'):  0.3888
  ('pain level', 'moderate'):  0.3389
  ('family communication', 'family called'):  0.3035
  ('restraint type', 'soft limb'):  0.2744
  ('riker-sas scale', 'very sedated'):  0.2585
  ('pain level', 'moderate to severe'):  0.2441
  ('verbal response', '1.0 et/trach'):  0.2343
  ('education barrier', 'medicated'):  0.2331
  ('reason for restraint', 'treatment interference'):  0.2314
  ('pain present', 'yes')  :  0.2312
  ('skin care', 'done')    :  0.2181
  ('orientation', 'oriented x 3'):  0.2180
  ('restraints